来自：https://github.com/pytorch/examples/tree/master/word_language_model 

导入需要使用到的包

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

这里还有一个`data_utils.py`文件，先看下文件中的内容。这里面主要是有两个class的定义：
 - Dictionary：字典，文字与序号的正反变换
 - Corpus：语料，将文字转化为数字，变换关系按照Dictionary进行

In [6]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)

In [7]:
import torch
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # 添加词语到字典中
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        # 标记解析文件，将所有的文字转化为数字，对应关系按照Dictionary执行
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

配置超参数

In [18]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000     # 取多少个点用于单次的LSTM输入
batch_size = 20
seq_length = 30
learning_rate = 0.002
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


加载数据集

In [10]:
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

创建RNN网络

In [12]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        # 词向量嵌入层
        self.embed = nn.Embedding(vocab_size, embed_size)
        # LSTM层
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        # 全连接层
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # 词向量嵌入层
        x = self.embed(x)
        
        # LSTM计算
        out, (h, c) = self.lstm(x, h)
        
        # 将输出转化为 (batch_size*sequence_length, hidden_size)，注意这里不再使用最后一个节点输出了
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # 全连接
        out = self.linear(out)
        return out, (h, c)

创建模型

In [15]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

训练模型

In [17]:
# 训练loop
for epoch in range(num_epochs):
    # 设置状态量
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # 获取输入/输出的batch
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # 前向传递
        states = [state.detach() for state in states] 
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # 参数更新
        optimizer.zero_grad()
        loss.backward()
        # 梯度裁剪，防止异常点对模型较大的影响
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))


Epoch [1/5], Step[0/1549], Loss: 2.8639, Perplexity: 17.53
Epoch [1/5], Step[100/1549], Loss: 2.7045, Perplexity: 14.95
Epoch [1/5], Step[200/1549], Loss: 2.9154, Perplexity: 18.46
Epoch [1/5], Step[300/1549], Loss: 2.7423, Perplexity: 15.52
Epoch [1/5], Step[400/1549], Loss: 2.8922, Perplexity: 18.03
Epoch [1/5], Step[500/1549], Loss: 2.4438, Perplexity: 11.52
Epoch [1/5], Step[600/1549], Loss: 2.9065, Perplexity: 18.29
Epoch [1/5], Step[700/1549], Loss: 2.7080, Perplexity: 15.00
Epoch [1/5], Step[800/1549], Loss: 2.8690, Perplexity: 17.62
Epoch [1/5], Step[900/1549], Loss: 2.4707, Perplexity: 11.83
Epoch [1/5], Step[1000/1549], Loss: 2.6458, Perplexity: 14.10
Epoch [1/5], Step[1100/1549], Loss: 2.7390, Perplexity: 15.47
Epoch [1/5], Step[1200/1549], Loss: 2.7593, Perplexity: 15.79
Epoch [1/5], Step[1300/1549], Loss: 2.6159, Perplexity: 13.68
Epoch [1/5], Step[1400/1549], Loss: 2.1874, Perplexity:  8.91
Epoch [1/5], Step[1500/1549], Loss: 2.6588, Perplexity: 14.28
Epoch [2/5], Step[0/

测试模型

In [ ]:
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # 初始化状态量
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # 随机选择一个词
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # 前向计算RNN
            output, state = model(input, state)

            # 计算输出的指数
            prob = output.exp()
            # 对输出进行采样，返回索引（选择最大的，返回索引？）
            word_id = torch.multinomial(prob, num_samples=1).item()

            # 填充新的词到input的后面
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
